In [63]:
import numpy as np
import pandas as pd
from load_data import load_train_data
from lda import train_lda, predict
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

feat,params,daq = load_train_data('train_data_raw_AB.mat')

In [13]:
# w1,c1 = lda.train_lda(feat,params[:,4,np.newaxis])
w,c = train_lda2(feat.T,params[:,4,np.newaxis].T)

In [14]:
# out1 = lda.predict(feat,w1,c1)
out = predict2(feat.T,w,c)

In [95]:
model = Sequential([
    Dense(units=60, input_shape=(60,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=7, activation='softmax')
])

In [71]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 60)                3660      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                1952      
_________________________________________________________________
dense_17 (Dense)             (None, 7)                 231       
Total params: 5,843
Trainable params: 5,843
Non-trainable params: 0
_________________________________________________________________


In [96]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [97]:
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

sub = 2
grp = 1
ind = (params[:,0] < sub) & (params[:,3] == grp)
feat_s, label_s = shuffle(feat[ind,:],params[ind,-2,np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(feat_s, label_s, test_size=0.3, random_state=0, stratify=label_s)
y_cat = to_categorical(y_train-1)
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train_n = scaler.transform(X_train)

1

In [98]:
model.fit(x=X_train_n, y=y_cat, validation_split=0.1, batch_size=32, epochs=30, verbose=2)

Epoch 1/30
69/69 - 0s - loss: 1.9028 - accuracy: 0.2281 - val_loss: 1.8942 - val_accuracy: 0.2816
Epoch 2/30
69/69 - 0s - loss: 1.7991 - accuracy: 0.3669 - val_loss: 1.7917 - val_accuracy: 0.4082
Epoch 3/30
69/69 - 0s - loss: 1.6939 - accuracy: 0.5002 - val_loss: 1.6800 - val_accuracy: 0.5265
Epoch 4/30
69/69 - 0s - loss: 1.5808 - accuracy: 0.6299 - val_loss: 1.5587 - val_accuracy: 0.6204
Epoch 5/30
69/69 - 0s - loss: 1.4553 - accuracy: 0.7252 - val_loss: 1.4298 - val_accuracy: 0.7184
Epoch 6/30
69/69 - 0s - loss: 1.3173 - accuracy: 0.7973 - val_loss: 1.2921 - val_accuracy: 0.7837
Epoch 7/30
69/69 - 0s - loss: 1.1773 - accuracy: 0.8395 - val_loss: 1.1525 - val_accuracy: 0.8571
Epoch 8/30
69/69 - 0s - loss: 1.0430 - accuracy: 0.8617 - val_loss: 1.0217 - val_accuracy: 0.8612
Epoch 9/30
69/69 - 0s - loss: 0.9208 - accuracy: 0.8830 - val_loss: 0.9041 - val_accuracy: 0.8735
Epoch 10/30
69/69 - 0s - loss: 0.8104 - accuracy: 0.9020 - val_loss: 0.7987 - val_accuracy: 0.8939
Epoch 11/30
69/69 -

In [121]:
w,c = train_lda(X_train_n,y_train)
X_test_n = scaler.transform(X_test)
out = predict(X_test_n,w,c)
np.sum(out.reshape(y_test.shape) == y_test-1)/y_test.shape[0]

0.9933333333333333